# Домашнее задание номер 3

Возьмите датасет с токсичными / нейтральными комментариями на русском языке:
https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments/data
Разделите датасет на обучающую и тестовую выборки. В качестве тестовой выборки
отделите 10% от всех данных.
Обучите бинарный классификатор на обучающей выборке, замерьте метрику F-score на
тестовой выборке.
Вы можете использовать как линейные модели, так и дообучать трансформеры. Можно
обучить несколько разных классификаторов и сравнить их качество

### Решение:

#### Датасет

1. Загрузка и очистка датасета

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('input/labeled.csv')
data = data.dropna()
data_filtered = data.loc[data['toxic'].isin({0, 1})]
df = data_filtered
df = df.reset_index(drop=True)
df.loc[(df['toxic'] == 0, 'Sentiment')] = 'positive'
df.loc[(df['toxic'] == 1, 'Sentiment')] = 'negative'
df = df.loc[df['comment'].apply(lambda text: len(text) >= 20)]
print(df['Sentiment'].value_counts())


Sentiment
positive    9586
negative    4826
Name: count, dtype: int64


2. Разделение на обучающие и тестовые группы

In [2]:
from sklearn.model_selection import train_test_split

texts = df['comment'].values
labels = df['Sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=0.1,
    random_state=1,
)
print(f'len(X_train) = {len(X_train)}')
print(f'len(y_train) = {len(y_train)}')
print(f'len(X_test) = {len(X_test)}')
print(f'len(y_test) = {len(y_test)}')

len(X_train) = 12970
len(y_train) = 12970
len(X_test) = 1442
len(y_test) = 1442


#### 2.Использование предварительно подготовленнои модель трансформера HuggingFace
Модель: https://huggingface.co/cointegrated/rubert-tiny-sentiment-balanced
У нее 12 миллионов параметродель

1. Подготовка токенизатора и модели

In [3]:
!pip install transformers sentencepiece --quiet

In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_pretrained = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model_pretrained.cuda()

C:\Users\Tabievaa\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2. Определение независимой от модели функции, которая будет возвращать прогноз для одного текста

In [6]:
from typing_extensions import Literal  
# from typing import Literal  # in Python 3.8+

def predict(
    model: AutoModelForSequenceClassification, 
    text: str, 
    return_type: Literal['label', 'score', 'proba'] = 'label',
):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

3. Тесты функции

In [7]:
textes = ['Очень рад, что смог помочь!', 'Ну кто же так делает?', 'Очень не очень']
for text in textes:
    # classify the text
    print(text)
    print(predict(model_pretrained, text, 'label'))  # negative
    # score the text on the scale from -1 (very negative) to +1 (very positive)
    print(predict(model_pretrained, text, 'score'))  # -0.5894946306943893
    # calculate probabilities of all labels
    print(predict(model_pretrained, text, 'proba'))  # [0.7870447  0.4947824  0.19755007]

Очень рад, что смог помочь!
positive
0.9454245362430811
[0.02615919 0.4202152  0.9715837 ]
Ну кто же так делает?
neutral
-0.22903871908783913
[0.2757948  0.97148424 0.04675609]
Очень не очень
neutral
-0.39927227050065994
[0.5195271  0.8157217  0.12025481]


4. Определение независимой от модели функции для оценки на тестовом наборе

In [8]:
from typing import Optional

from sklearn.metrics import classification_report
from tqdm.auto import tqdm

def evaluate_model(
    model: AutoModelForSequenceClassification, 
    subset: Optional[int] = None,
):
    y_pred = []
    
    if subset is None:
        subset = X_test.shape[0]
    
    for x in tqdm(X_test[:subset]):
        prediction = predict(model, x)
        y_pred.append(prediction)
        
    print(classification_report(y_test[:subset], y_pred))

In [9]:
evaluate_model(model_pretrained)

  0%|          | 0/1442 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.51      0.62      0.56       494
     neutral       0.00      0.00      0.00         0
    positive       0.75      0.12      0.20       948

    accuracy                           0.29      1442
   macro avg       0.42      0.25      0.25      1442
weighted avg       0.67      0.29      0.32      1442



C:\Users\Tabievaa\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tabievaa\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tabievaa\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
(0.56 + 0.20) / 2

0.38

#### 3. Доработка предварительно обученной модели на основе нашего набора данных

1. Разделение обучающих данных на train и dev

In [11]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [12]:
from datasets import Dataset, DatasetDict

n_train_examples = 10000
n_dev_examples = 5000
train_df = pd.DataFrame(
    {
        'text': X_train[:n_train_examples],
        'label': y_train[:n_train_examples],
    }
)
dev_df = pd.DataFrame(
    {
        'text': X_train[n_train_examples:n_train_examples + n_dev_examples],
        'label': y_train[n_train_examples:n_train_examples + n_dev_examples],
    }
)

data = DatasetDict(
    {
        'train': Dataset.from_pandas(train_df[['text', 'label']].reset_index(drop=True)),
        'dev': Dataset.from_pandas(dev_df[['text', 'label']].reset_index(drop=True)),
    }
)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 2970
    })
})

2. Маркировка данных

In [13]:
all_labels = ['negative', 'neutral', 'positive']
data_tokenized = data.map(
    lambda row: tokenizer(row['text'], truncation=True), batched=True, remove_columns=['text']
)
data_tokenized = data_tokenized.map(
    lambda row: {'label': [all_labels.index(label) for label in row['label']]}, batched=True
)
data_tokenized

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2970 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2970 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2970
    })
})

3. Подготовка к обучению

In [14]:
!pip install evaluate

In [16]:
import evaluate
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer)

training_args = TrainingArguments(
    output_dir='test_trainer', 
    evaluation_strategy='epoch',
    report_to = None
)
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
        predictions=predictions, 
        references=labels,
    )

trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=data_tokenized['train'],
    eval_dataset=data_tokenized['dev'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

4. Обучение и сохранение модели

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.351700,0.290357,0.891919
2,0.243600,0.372831,0.897643
3,0.172000,0.419705,0.898990


TrainOutput(global_step=3750, training_loss=0.26211282755533855, metrics={'train_runtime': 3960.5008, 'train_samples_per_second': 7.575, 'train_steps_per_second': 0.947, 'total_flos': 74876167844064.0, 'train_loss': 0.26211282755533855, 'epoch': 3.0})

In [19]:
trainer.save_model('my_homework_model')

5. Загрузка модели

In [20]:
checkpoint = 'my_homework_model'
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny-sentiment-balanced')
model_finetuned = AutoModelForSequenceClassification.from_pretrained(checkpoint)
if torch.cuda.is_available():
    model_finetuned.cuda()

C:\Users\Tabievaa\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
examples = [
    'Рад, что удалось помочь!',
    'Как же все херово',
    'Пощады, сапсите, помогите',
    'У Васи ужасное настроение.',
    'А что в итоге надо делать?))'
]
for example in examples:
    print(predict(model_finetuned, example))

positive
negative
negative
negative
positive


6. Оценка

In [24]:
print(checkpoint)
evaluate_model(model_finetuned)

my_homework_model


  0%|          | 0/1442 [00:00<?, ?it/s]

              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       494
    positive       0.92      0.92      0.92       948

    accuracy                           0.89      1442
   macro avg       0.88      0.88      0.88      1442
weighted avg       0.89      0.89      0.89      1442



In [25]:
(0.84 + 0.92) / 2

0.88

## Результат:
* В короткие сроки удалось обучить модель на датасете негативных отзывов на русском языке;
